In [1]:
#Before we can run this cope
#Install mlflow on local machine by pip3 install mlflow
#Start the server
# mlflow server --backend-store-uri sqlite:///mydb.sqlitebackend_store --default-artifact-root /home/gaurav/Documents/SoftwareInstall/ml_flow/artifact_root --host 0.0.0.0


In [2]:
#first we set the tracking server
import mlflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("example")
mlflow.autolog()

2022/06/25 15:48:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.
2022/06/25 15:48:07 INFO mlflow.pyspark.ml: No SparkSession detected. Autologging will log pyspark.ml models contained in the default allowlist. To specify a custom allowlist, initialize a SparkSession prior to calling mlflow.pyspark.ml.autolog() and specify the path to your allowlist file via the spark.mlflow.pysparkml.autolog.logModelAllowlistFile conf.
2022/06/25 15:48:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.


In [3]:
#now some ML code
import numpy as np
import pandas as pd
#Lets load the data
col_name = ['sepal-lenght', 'sepal-width',
            'petal-lenght', 'petal-width', 'class']

df = pd.read_csv('./iris.csv', names=col_name)

df.head


<bound method NDFrame.head of      sepal-lenght  sepal-width  petal-lenght  petal-width           class
0             5.1          3.5           1.4          0.2     Iris-setosa
1             4.9          3.0           1.4          0.2     Iris-setosa
2             4.7          3.2           1.3          0.2     Iris-setosa
3             4.6          3.1           1.5          0.2     Iris-setosa
4             5.0          3.6           1.4          0.2     Iris-setosa
..            ...          ...           ...          ...             ...
145           6.7          3.0           5.2          2.3  Iris-virginica
146           6.3          2.5           5.0          1.9  Iris-virginica
147           6.5          3.0           5.2          2.0  Iris-virginica
148           6.2          3.4           5.4          2.3  Iris-virginica
149           5.9          3.0           5.1          1.8  Iris-virginica

[150 rows x 5 columns]>

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# we will run our code on multiple models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier(
    n_estimators=3, max_depth=5, random_state=0)))
models.append(('NB', GaussianNB()))
models.append(('SVC', SVC(gamma='auto')))

results = []
model_names = []


2022/06/25 15:48:17 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [5]:
from sklearn.model_selection import train_test_split

X = df[['sepal-lenght', 'sepal-width',
        'petal-lenght', 'petal-width']]
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=1)


In [6]:
#For each model we have

for name, model in models:
    mlflow.start_run()
    mlflow.sklearn.autolog()
    #we want to do k fold cross validation
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(
        model, X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    model_names.append(name)

    model.fit(X_train, y_train)
    accuracy = accuracy_score(y_test, model.predict(X_test))
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
    
    #Lets track some parameters and metric
    mlflow.log_param("Algorithm_Name", name)
    mlflow.log_metric("Score mean", cv_results.mean())

    mlflow.log_metric("Score std deviation", cv_results.std())

    mlflow.log_metric("accuracy", accuracy)
    
    #Next lets save our dataframe for x and y and store them as artifacts
    mlflow.log_artifact('./iris.csv')    
    #finally lets store our modle
    mlflow.sklearn.log_model(model,"model")
    mlflow.end_run()


/home/gaurav/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/gaurav/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

LR: 0.966667 (0.040825)


2022/06/25 15:48:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/gaurav/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator."


LDA: 0.975000 (0.038188)


/home/gaurav/.local/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/gaurav/.local/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/gaurav/.local/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/gaurav/.local/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/gaurav/.local/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/gaurav/.local/lib/python3.8/site-packag

...

RF: 0.950000 (0.040825)


2022/06/25 15:48:44 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/gaurav/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator."


NB: 0.950000 (0.055277)


2022/06/25 15:48:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/gaurav/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator."


SVC: 0.983333 (0.033333)
